<center>
<img src='./img/nsidc_logo.png'/>

# **IceFlow**
### Point Cloud Data Access
</center>

### NASA's EarthData Credentials

The first step to start working with IceFlow data is to login into [NASA's Earthdata Search](https://earthdata.nasa.gov/).

> **Note**: you don't need to be a NASA employee to register with NASA EarthData!


In [1]:
# We import our IceFlow client library
from iceflow.client import IceflowClient
# We instantiate our client
iceflow = IceflowClient()
# We need to use our NASA Earth Data Credentials and verify that they work.
# Please click on set credentials and then see if we are authenticated by executing the next cell.
iceflow.display(['credentials'])

Text(value='', description='User:', placeholder='Your NASA Earth username ')

Password(description='Password:', placeholder='Enter password')

Button(description='Set Credentials', style=ButtonStyle())

In [2]:
# We are going to verify that our credentials are valid.
session = iceflow.create_earthdata_authenticated_session()
if session is None:
    print('we are not logged into NASA EarthData')
else:
    print('we are logged into NASA EarthData!')

we are logged into NASA EarthData!


In [4]:
# Let's start with the user interface. We'll explain what this does next.
# vertical = Sidecar widget, horizontal = render the widget in this notebook.
iceflow.display(['map','controls'], 'horizontal', extra_layers=True)

Output(layout=Layout(border='1px solid black'), outputs=({'output_type': 'display_data', 'data': {'text/plain'…

## Placing a data order for one of the data sets served by IceFlow

Now that we have our constraints we just need to post our order and wait for IceFlow to fulfill it. 

We can put an order directly, in this case we are going to work on a geometry that overlaps with [Jakobshavn](https://en.wikipedia.org/wiki/Jakobshavn_Glacier) glacier in Greenland or Thwaites Glacier in Antarctica (depending on which one you select). 




In [9]:
# note that we are explicitly using the name of one of the ATM datasets
# dataset = 'ILATM1B'
dataset = 'GLAH06'

# Thwaites glacier
my_params ={
    'dataset': 'GLAH06',
    'start': '2001-01-01',
    'end': '2011-1-01',
    'bbox': '-107.4515,-75.3695,-105.3794,-74.4563'
}

# Jakobshavn 2016
# my_params ={
#     'dataset': dataset,
#     'start': '2016-01-01',
#     'end': '2016-12-31',
#     'bbox': '-107.4515,-75.3695,-105.3794,-74.4563'
# }

# returns a json dictionary, the request parameters and the order's response.
order = iceflow.post_iceflow_order(my_params)
order

{'request': {'bbox': '-107.4515,-75.3695,-105.3794,-74.4563',
  'time_range': '2001-01-01,2011-1-01'},
 'response': <Response [200]>}

## Downloading the data
Let's get some coffee, some IceFlow orders are in the Gigabytes real and may take a little while to be processed. 
Once that your status URL says is completed we can grab the HDF5 data file using the URL on the same response!

The first thing we need to know is the order status

In [10]:
order['response'].json()

{'message': 'Order queued for processing.',
 'order_id': '88a1e685-b7a4-443a-8f48-922836ce82f9',
 'status_url': 'http://valkyrie-vm.apps.nsidc.org/1.0/order-status/88a1e685-b7a4-443a-8f48-922836ce82f9'}

In [11]:
order_status = iceflow.order_status(order)
if 'url' in order_status:
    data_url = order_status['url']
order_status

{'message': 'Order is complete. Data now available for download.',
 'order_id': '88a1e685-b7a4-443a-8f48-922836ce82f9',
 'status': 'COMPLETE',
 'url': 'http://valkyrie-vm.apps.nsidc.org/1.0/order/88a1e685-b7a4-443a-8f48-922836ce82f9'}

## Data order is complete
Now that we know the order is complete we can download the file in our status response `url` to our local working directory using requests or curl or wget or just clicking the link.



In [12]:
# We can name our file or leave it with the original file name by not using the file_name parameter.
file_name = 'twaties-test-GLAH06-2000-2010'
iceflow_file = iceflow.download_order(data_url, file_name)

## Placing multiple data orders to IceFlow

If we need more than one data set from IceFlow at the same time, we can use the client to place our data orders, IceFlow will return a single HDF5 file for each of the data sets that we have selected using the map widget.

In [ ]:
# orders = v.post_orders()
# orders